In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [2]:
#数据加载
data = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "occupation", "zip"]
dense_features = ["age"]
target = ['rating']

In [3]:
# 对类别特征进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [4]:
# 对数值特征进行归一化
mm = MinMaxScaler()
data[dense_features] = mm.fit_transform(data[dense_features])

In [5]:
# 生成feature columns
Sparse_feature = [SparseFeat(feature, vocabulary_size=data[feature].nunique()) for feature in sparse_features]
Dense_feature = [DenseFeat(feature, dimension=1) for feature in dense_features]

fixlen_feature_columns = Sparse_feature + Dense_feature
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [6]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}    
test_model_input = {name:test[name].values for name in feature_names}

In [7]:
# DeepFM 定义、配置、训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=128, epochs=50, verbose=True, validation_split=0.2,)

Epoch 1/50


e:\tool\python36\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 282ms/step - loss: 15.2769 - mse: 15.2769 - val_loss: 17.1445 - val_mse: 17.1445
Epoch 2/50
1/1 [==============================] - 0s 16ms/step - loss: 14.9251 - mse: 14.9251 - val_loss: 16.7856 - val_mse: 16.7856
Epoch 3/50
1/1 [==============================] - 0s 17ms/step - loss: 14.5808 - mse: 14.5808 - val_loss: 16.4372 - val_mse: 16.4372
Epoch 4/50
1/1 [==============================] - 0s 15ms/step - loss: 14.2486 - mse: 14.2486 - val_loss: 16.0949 - val_mse: 16.0949
Epoch 5/50
1/1 [==============================] - 0s 18ms/step - loss: 13.9241 - mse: 13.9241 - val_loss: 15.7631 - val_mse: 15.7631
Epoch 6/50
1/1 [==============================] - 0s 16ms/step - loss: 13.6104 - mse: 13.6104 - val_loss: 15.4371 - val_mse: 15.4371
Epoch 7/50
1/1 [==============================] - 0s 16ms/step - loss: 13.3001 - mse: 13.3001 - val_loss: 15.1119 - val_mse: 15.1119
Epoch 8/50
1/1 [==============================] - 0s 17ms/step - loss: 12.9887 

In [8]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)

In [9]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)          # 从第四位小数，四舍五入
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 1.1779218989389746
